In [1]:
from dbfread import DBF
import pandas as pd

file_path = './data_raw/defunciones_base_datos_2023_dbf/DEFUN23.dbf' # <-- pega el path  al folder dentro de las comillas

df = DBF(file_path) #cargar archivo dbf
df = pd.DataFrame(df)# volver DataFrame
df.tail()

,ENT_REGIS,MUN_REGIS,TLOC_REGIS,LOC_REGIS,ENT_RESID,MUN_RESID,TLOC_RESID,LOC_RESID,ENT_OCURR,MUN_OCURR,...,COMPLICARO,DIA_CERT,MES_CERT,ANIO_CERT,MATERNAS,ENT_OCULES,MUN_OCULES,LOC_OCULES,RAZON_M,DIS_RE_OAX
799864,32,024,8,0001,32,024,8,0001,32,024,...,8,99,12,2023,,88,888,8888,NaN,999
799865,32,056,13,0001,32,056,13,0001,32,056,...,8,5,6,2023,,88,888,8888,NaN,999
799866,32,056,13,0001,32,017,4,0042,32,056,...,8,9,8,2023,,88,888,8888,NaN,999
799867,32,056,13,0001,32,017,13,0001,32,056,...,8,19,9,2023,,88,888,8888,NaN,999
799868,32,038,5,0001,32,038,1,0075,32,038,...,8,6,4,2023,,88,888,8888,NaN,999


# Seleccionar


In [2]:
columns = ['ENT_OCURR', 'MUN_OCURR', 'LOC_OCURR', 'AREA_UR', 
            'SEXO', 'EDAD', 'EDAD_AGRU', 'DIA_OCURR', 'MES_OCURR', 'ANIO_OCUR', 
            'CAUSA_DEF', 'TIPO_DEFUN', 'LUGAR_OCUR', 
            'SITIO_OCUR', 'COND_CERT', 'DERECHOHAB', ]
df = df[columns]
df.tail()

,ENT_OCURR,MUN_OCURR,LOC_OCURR,AREA_UR,SEXO,EDAD,EDAD_AGRU,DIA_OCURR,MES_OCURR,ANIO_OCUR,CAUSA_DEF,TIPO_DEFUN,LUGAR_OCUR,SITIO_OCUR,COND_CERT,DERECHOHAB
799864,32,024,0001,1,2,2021,01,22,12,2023,P249,4,88,11,3,99
799865,32,056,0001,1,2,3001,01,5,6,2023,P579,4,88,3,1,2
799866,32,056,0001,1,2,1003,01,9,8,2023,P220,4,88,3,1,2
799867,32,056,0001,1,2,2007,01,19,9,2023,Q793,4,88,3,1,2
799868,32,038,0001,2,1,1023,01,5,4,2023,P285,4,88,2,3,1


In [3]:
def map_cie10_diagnostics_generalized(df, catalog, df_cie_col, catalog_cie_col):
    """
    Maps CIE-10 diagnostics from a dataframe to a catalog, handling both 3- and 4-digit codes.

    Args:
        df (pd.DataFrame): The main dataframe.
        catalog (pd.DataFrame): The catalog dataframe.
        df_cie_col (str): The name of the column in `df` with CIE-10 codes (4 digits).
        catalog_cie_col (str): The name of the column in `catalog` with CIE-10 codes (3 or 4 digits).

    Returns:
        pd.DataFrame: The original dataframe with the additional columns from the
                      catalog joined.
    """
    # Create a 3-digit code column in the catalog for broader matching
    catalog_3digit_col = f"{catalog_cie_col}_3digit"
    catalog[catalog_3digit_col] = catalog[catalog_cie_col].str[:3]

    # First merge: Attempt a direct match on the full 4-digit codes.
    df_mapped = pd.merge(df, catalog, left_on=df_cie_col, right_on=catalog_cie_col, how='left')

    # Find the rows that did not match in the first merge
    unmatched_df = df_mapped[df_mapped[catalog_cie_col].isnull()]
    matched_df = df_mapped[df_mapped[catalog_cie_col].notnull()]

    if not unmatched_df.empty:
        # Create a 3-digit code from the main dataframe for the second merge
        df_3digit_col = f"{df_cie_col}_3digit"
        unmatched_df[df_3digit_col] = unmatched_df[df_cie_col].str[:3]

        # Merge the unmatched rows on the 3-digit codes.
        merged_unmatched = pd.merge(
            unmatched_df.drop(columns=[catalog_cie_col, catalog_3digit_col]),
            catalog,
            left_on=df_3digit_col,
            right_on=catalog_3digit_col,
            how='left',
            suffixes=('_orig', '_new')
        )

        # Combine the successfully matched rows with the newly matched rows.
        final_df = pd.concat([matched_df, merged_unmatched])

        final_df = final_df.drop(columns=[df_3digit_col, catalog_3digit_col])
        # Join 3 and 4 digit merges and clean up temporary columns.
        for col in catalog.drop(columns=[catalog_cie_col, catalog_3digit_col]).columns:
            final_df[col] = final_df[col].fillna(final_df[f"{col}_orig"]).fillna(final_df[f"{col}_new"])
            final_df = final_df.drop(columns=[f"{col}_orig", f"{col}_new"])
            
    else:
        final_df = df_mapped
        final_df = final_df.drop(columns=[catalog_3digit_col]).reset_index(drop=True)

    return final_df

file_path = './data_raw/EnfermedadesRespiratorias_CIE10.csv'
cat_respi = pd.read_csv(file_path)
display(cat_respi.tail())

df = map_cie10_diagnostics_generalized(df, cat_respi, df_cie_col='CAUSA_DEF', catalog_cie_col='CodigoCIE')
df = df.drop(columns=['CodigoCIE', 'TerminoCIE'])
df = df[df['ClasificacionDiagnostico'].notna()].rename({'ClasificacionDiagnostico':'CLA_DIAG'}, axis=1)
df['CLA_DIAG'] = df['CLA_DIAG'].astype('category')
df.index = range(df.shape[0])
df

,CodigoCIE,TerminoCIE,ClasificacionDiagnostico
119,C38,"Tumor maligno del corazón, del mediastino y de...",Tumores malignos del tórax (C30-C40)
120,C39,Tumor maligno de otros sitios y de los mal def...,Tumores malignos del tórax (C30-C40)
121,C40,Tumor maligno de los huesos y de los cartílago...,Tumores malignos del tórax (C30-C40)
122,C761,Tumor maligno del tórax,Tumores torácicos mal especificados (C76.1)
123,B206,"Enfermedad por VIH, resultante en neumonía por...",VIH y neumonía por Pneumocystis jirovecii o co...


/tmp/ipykernel_6682/2357838224.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_df[df_3digit_col] = unmatched_df[df_cie_col].str[:3]


,ENT_OCURR,MUN_OCURR,LOC_OCURR,AREA_UR,SEXO,EDAD,EDAD_AGRU,DIA_OCURR,MES_OCURR,ANIO_OCUR,CAUSA_DEF,TIPO_DEFUN,LUGAR_OCUR,SITIO_OCUR,COND_CERT,DERECHOHAB,CLA_DIAG
0,01,001,0001,1,2,4077,20,12,12,2022,J80X,4,88,1,1,1,Síndrome de dificultad respiratoria aguda (J80)
1,01,001,0001,1,1,4052,15,18,1,2023,J64X,4,88,3,1,2,Daño pulmonar por agentes externos (J60-J70)
2,01,005,0001,1,1,4095,24,18,1,2023,U071,4,88,3,1,2,"COVID-19 (U071, U072)"
3,01,005,0001,1,1,4062,17,7,1,2023,U071,4,88,3,3,2,"COVID-19 (U071, U072)"
4,01,001,0001,1,1,4032,11,11,1,2023,U071,4,88,1,1,1,"COVID-19 (U071, U072)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96555,32,056,0001,2,1,1097,01,11,3,2023,P251,4,88,3,1,2,"Hipoxia neonatal, aspiración, neumonía neonata..."
96556,32,055,0001,2,1,1012,01,19,6,2023,P209,4,88,2,1,1,"Hipoxia neonatal, aspiración, neumonía neonata..."
96557,32,024,0001,1,2,2021,01,22,12,2023,P249,4,88,11,3,99,"Hipoxia neonatal, aspiración, neumonía neonata..."
96558,32,056,0001,1,2,1003,01,9,8,2023,P220,4,88,3,1,2,"Hipoxia neonatal, aspiración, neumonía neonata..."


# Mapear

In [4]:
dicts_replace = {
        'AREA_UR': {1:'Urbana', 2:'Rural', 9:'No especificada'},
        'SEXO':    {1:'Hombre', 2:'Mujer', 9:'No especificado'},
        'TIPO_DEFUN': {1:'Accidente', 2:'Homicidio (Agresión)', 3:'Suicidio (Lesión autoinfligida)', 
                       4:'Enfermedad (Muerte natural)', 5:'Intervención legal', 6:'Se ignora', 9:'No especificado'},
        'LUGAR_OCUR': {0:'Vivienda particular', 1:'Vivienda colectiva', 2:'Escuela u oficina pública', 
                       3:'Áreas deportivas', 4:'Calle o carretera (vía pública)', 
                       5:'Área comercial o de servicios', 6:'Área industrial (taller, fabrica u obra)', 
                       7:'Granja (rancho o parcela)', 8:'Otro', 9:'Se ignora', 88:'No aplica para muerte natural'},
        'SITIO_OCUR': {1:'Secretaría de Salud', 2:'IMSS BIENESTAR', 3:'IMSS', 4:'ISSSTE', 5:'PEMEX', 
                       6:'Secretaría de la Defensa Nacional (SEDENA)', 7:'Secretaría de Marina (SEMAR)', 
                       8:'Otra unidad pública', 9:'Unidad médica privada', 10:'Vía pública', 11:'Hogar', 
                       12:'Otro lugar', 99:'No especificado'},
        'COND_CERT':  {1:'Médico tratante', 2:'Médico legista', 3:'Otro médico', 4:'Persona autorizada por la SSA', 
                       5:'Autoridad civil', 8:'Otra', 9:'No especificada'},
        'DERECHOHAB': {1:'Ninguna', 2:'IMSS', 3:'ISSSTE', 4:'PEMEX', 5:'SEDENA', 6:'SEMAR', 
                       7:'Reportado como equivalente al Seguro Popular', 8:'Otra', 9:'IMSS BIENESTAR', 
                       10:'Instituto de Seguridad Social para las Fuerzas Armadas Mexicanas (ISSFAM)', 99:'No especificada'},
}

for col, dic_replace in dicts_replace.items():
    df[col] = df[col].replace(dic_replace)
    df[col] = df[col].astype('category')
    display(df[col].value_counts())


AREA_UR
Urbana             78135
Rural              16880
No especificada     1545
Name: count, dtype: int64

SEXO
Hombre             55575
Mujer              40970
No especificado       15
Name: count, dtype: int64

TIPO_DEFUN
Enfermedad (Muerte natural)        94799
Accidente                           1662
No especificado                       85
Suicidio (Lesión autoinfligida)       14
Name: count, dtype: int64

LUGAR_OCUR
No aplica para muerte natural               94799
Otro                                          908
Se ignora                                     302
Calle o carretera (vía pública)               245
Vivienda particular                           162
Granja (rancho o parcela)                      93
Área comercial o de servicios                  29
Áreas deportivas                                8
Escuela u oficina pública                       7
Área industrial (taller, fabrica u obra)        4
Vivienda colectiva                              3
Name: count, dtype: int64

SITIO_OCUR
Hogar                                         32551
IMSS                                          25360
Secretaría de Salud                           18213
Unidad médica privada                          5483
ISSSTE                                         4563
Otro lugar                                     3764
Otra unidad pública                            1789
IMSS BIENESTAR                                 1443
No especificado                                1296
Vía pública                                     993
PEMEX                                           552
Secretaría de la Defensa Nacional (SEDENA)      438
Secretaría de Marina (SEMAR)                    115
Name: count, dtype: int64

COND_CERT
Otro médico                      60789
Médico tratante                  21533
Médico legista                    6970
No especificada                   4191
Otra                              1730
Persona autorizada por la SSA      712
Autoridad civil                    635
Name: count, dtype: int64

DERECHOHAB
IMSS                                                                         37736
Ninguna                                                                      33757
No especificada                                                              11348
ISSSTE                                                                        7394
Otra                                                                          2956
IMSS BIENESTAR                                                                1528
PEMEX                                                                          830
SEDENA                                                                         372
Instituto de Seguridad Social para las Fuerzas Armadas Mexicanas (ISSFAM)      327
Reportado como equivalente al Seguro Popular                                   161
SEMAR                                                                          151
Name: count, dtype: int64

In [5]:
dic_replace = {99:pd.NA,9999:pd.NA}
for col in ['DIA_OCURR', 'MES_OCURR', 'ANIO_OCUR']:
    df[col] = df[col].replace(dic_replace)
    display(df[col].value_counts())

data = df[['ANIO_OCUR', 'MES_OCURR', 'DIA_OCURR']]
data.columns = ['year', 'month', 'day']

df['FECHA_OCURR'] = pd.to_datetime(data, errors='coerce')

df = df[df['ANIO_OCUR']==2023]
df.shape

DIA_OCURR
17    3296
12    3294
16    3289
18    3234
26    3224
27    3220
21    3214
22    3212
24    3212
23    3210
25    3200
5     3182
7     3177
13    3173
1     3172
14    3171
4     3168
3     3164
6     3163
15    3161
2     3142
10    3135
8     3132
20    3122
28    3118
11    3109
19    3091
9     3022
29    2842
30    2817
31    1866
Name: count, dtype: int64

MES_OCURR
1     11187
12     9997
2      8042
3      7966
11     7947
6      7877
4      7476
10     7391
5      7373
9      7159
7      7070
8      7055
Name: count, dtype: int64

ANIO_OCUR
2023    94268
2022     1879
2021      131
2020       92
2019       39
2018       32
2017       19
2015       15
2016       11
2013       10
2008        7
2014        7
2011        6
2005        4
2004        3
2003        3
2010        3
2009        3
1989        2
2002        2
1997        2
2007        2
1995        1
2000        1
2006        1
1965        1
1969        1
1963        1
1975        1
1992        1
1999        1
1985        1
1988        1
2012        1
1961        1
1998        1
Name: count, dtype: int64

(94268, 18)

In [6]:
def modify_age_inegi(number):
    if number % 100 == 98: return pd.NA
    elif number<=4000: return 0
    else: return number-4000

df['EDAD'] = df['EDAD'].apply(modify_age_inegi)

/tmp/ipykernel_6682/329356222.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EDAD'] = df['EDAD'].apply(modify_age_inegi)


In [7]:
df['EDAD_AGRU'].value_counts()

EDAD_AGRU
21    11144
20    10634
22    10239
19     9068
18     7626
23     6639
01     6213
17     6163
16     4758
15     3792
14     2951
24     2565
13     2544
12     2132
11     1952
10     1579
09     1062
08      680
02      522
25      509
07      344
06      321
03      251
30      220
04      180
05      123
26       48
27        8
28        1
Name: count, dtype: int64

In [8]:
from pandas.api.types import CategoricalDtype

dic_replace = {'01':'Menores de un año', '02':'De un año', '03':'De 2', '04':'De 3', '05':'De 4', '06':'De 5 a 9', 
               '07':'De 10 a 14', '08':'De 15 a 19', '09':'De 20 a 24', '10':'De 25 a 29', '11':'De 30 a 34', 
               '12':'De 35 a 39', '13':'De 40 a 44', '14':'De 45 a 49', '15':'De 50 a 54', '16':'De 55 a 59', 
               '17':'De 60 a 64', '18':'De 65 a 69', '19':'De 70 a 74', '20':'De 75 a 79', '21':'De 80 a 84', 
               '22':'De 85 a 89', '23':'De 90 a 94', '24':'De 95 a 99', '25':'De 100 a 104', '26':'De 105 y más', 
               '27':'De 105 y más', '28':'De 105 y más', '29':'De 105 y más', '30':pd.NA}
df['EDAD_AGRU'] = df['EDAD_AGRU'].replace(dic_replace)

cat_ordered = list(dic_replace.values())[0:26]
cat_ordered = CategoricalDtype(categories=cat_ordered, ordered=True)
df['EDAD_AGRU'] = df['EDAD_AGRU'].astype(cat_ordered)

df['EDAD_AGRU'].value_counts().sort_index()

/tmp/ipykernel_6682/1130306981.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EDAD_AGRU'] = df['EDAD_AGRU'].replace(dic_replace)
/tmp/ipykernel_6682/1130306981.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EDAD_AGRU'] = df['EDAD_AGRU'].astype(cat_ordered)


EDAD_AGRU
Menores de un año     6213
De un año              522
De 2                   251
De 3                   180
De 4                   123
De 5 a 9               321
De 10 a 14             344
De 15 a 19             680
De 20 a 24            1062
De 25 a 29            1579
De 30 a 34            1952
De 35 a 39            2132
De 40 a 44            2544
De 45 a 49            2951
De 50 a 54            3792
De 55 a 59            4758
De 60 a 64            6163
De 65 a 69            7626
De 70 a 74            9068
De 75 a 79           10634
De 80 a 84           11144
De 85 a 89           10239
De 90 a 94            6639
De 95 a 99            2565
De 100 a 104           509
De 105 y más            57
Name: count, dtype: int64

In [9]:
file_path = 'data_raw/defunciones_base_datos_2023_dbf/CATEMLDE23.dbf'
cat_geo = DBF(file_path)
cat_geo = pd.DataFrame(cat_geo)

cat_geo['NOM_LOC'] = cat_geo['NOM_LOC'].replace({'Cifra confidencial. Establecido en la Ley del SNIEG.':'Confidencial'}) 
cat_geo['CVE_ENT'] = cat_geo['CVE_ENT'].replace({f"{s}{s}":pd.NA for s in '089'})
cat_geo['CVE_MUN'] = cat_geo['CVE_MUN'].replace({f"{s}{s}{s}":pd.NA for s in '089'})
cat_geo['CVE_LOC'] = cat_geo['CVE_LOC'].replace({f"{s}{s}{s}{s}":pd.NA for s in '089'})

cols_merge = ['CVE_ENT']
catalog = cat_geo.groupby(cols_merge)['NOM_LOC'].first() \
                 .reset_index().rename({'NOM_LOC':'NOM_ENT'}, axis=1)
df = pd.merge(df, catalog, left_on=['ENT_OCURR'], right_on=cols_merge, how='left').drop(columns=cols_merge)

cols_merge = ['CVE_ENT','CVE_MUN']
catalog = cat_geo.groupby(cols_merge)['NOM_LOC'].first() \
                 .reset_index().rename({'NOM_LOC':'NOM_MUN'}, axis=1)
df = pd.merge(df, catalog, left_on=['ENT_OCURR', 'MUN_OCURR'], right_on=cols_merge, how='left').drop(columns=cols_merge)

cols_merge = ['CVE_ENT', 'CVE_MUN', 'CVE_LOC']
df = pd.merge(df, cat_geo, left_on=['ENT_OCURR', 'MUN_OCURR', 'LOC_OCURR'], right_on=cols_merge, how='left').drop(columns=cols_merge)
df.tail()

,ENT_OCURR,MUN_OCURR,LOC_OCURR,AREA_UR,SEXO,EDAD,EDAD_AGRU,DIA_OCURR,MES_OCURR,ANIO_OCUR,...,TIPO_DEFUN,LUGAR_OCUR,SITIO_OCUR,COND_CERT,DERECHOHAB,CLA_DIAG,FECHA_OCURR,NOM_ENT,NOM_MUN,NOM_LOC
94263,32,056,0001,Rural,Hombre,0,Menores de un año,11,3,2023,...,Enfermedad (Muerte natural),No aplica para muerte natural,IMSS,Médico tratante,IMSS,"Hipoxia neonatal, aspiración, neumonía neonata...",2023-03-11,Zacatecas,Zacatecas,Zacatecas
94264,32,055,0001,Rural,Hombre,0,Menores de un año,19,6,2023,...,Enfermedad (Muerte natural),No aplica para muerte natural,IMSS BIENESTAR,Médico tratante,Ninguna,"Hipoxia neonatal, aspiración, neumonía neonata...",2023-06-19,Zacatecas,Villanueva,Villanueva
94265,32,024,0001,Urbana,Mujer,0,Menores de un año,22,12,2023,...,Enfermedad (Muerte natural),No aplica para muerte natural,Hogar,Otro médico,No especificada,"Hipoxia neonatal, aspiración, neumonía neonata...",2023-12-22,Zacatecas,Loreto,Loreto
94266,32,056,0001,Urbana,Mujer,0,Menores de un año,9,8,2023,...,Enfermedad (Muerte natural),No aplica para muerte natural,IMSS,Médico tratante,IMSS,"Hipoxia neonatal, aspiración, neumonía neonata...",2023-08-09,Zacatecas,Zacatecas,Zacatecas
94267,32,038,0001,Rural,Hombre,0,Menores de un año,5,4,2023,...,Enfermedad (Muerte natural),No aplica para muerte natural,IMSS BIENESTAR,Otro médico,Ninguna,"Hipoxia neonatal, aspiración, neumonía neonata...",2023-04-05,Zacatecas,Pinos,Pinos


In [10]:
df.columns

Index(['ENT_OCURR', 'MUN_OCURR', 'LOC_OCURR', 'AREA_UR', 'SEXO', 'EDAD',
       'EDAD_AGRU', 'DIA_OCURR', 'MES_OCURR', 'ANIO_OCUR', 'CAUSA_DEF',
       'TIPO_DEFUN', 'LUGAR_OCUR', 'SITIO_OCUR', 'COND_CERT', 'DERECHOHAB',
       'CLA_DIAG', 'FECHA_OCURR', 'NOM_ENT', 'NOM_MUN', 'NOM_LOC'],
      dtype='object')

In [11]:
cols = ['SEXO', 'EDAD', 'EDAD_AGRU', 
        'DIA_OCURR', 'MES_OCURR', 'ANIO_OCUR', 'FECHA_OCURR', 
        'ENT_OCURR', 'MUN_OCURR', 'LOC_OCURR', 'NOM_ENT', 'NOM_MUN', 'NOM_LOC', 
        'CAUSA_DEF', 'CLA_DIAG', 'TIPO_DEFUN', 'AREA_UR',
        'LUGAR_OCUR', 'SITIO_OCUR', 'COND_CERT', 'DERECHOHAB',
       ]
df = df[cols]

In [15]:
df.to_csv('data_clean/EDR2023_Respiratorias.csv', index=False)
df.to_pickle('data_clean/EDR2023_Respiratorias.pkl') 